In [3]:
import time

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import IPython.display as ipd

from scipy.io import wavfile as wav

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,LearningRateScheduler

import warnings
warnings.filterwarnings('ignore')

2022-11-01 13:01:59.716847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-01 13:02:00.121846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-01 13:02:00.121884: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-01 13:02:00.174342: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-01 13:02:01.200406: W tensorflow/stream_executor/platform/de

In [6]:
SEED = 42

import os
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import random 
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

# NLTK Vader

In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vds = SentimentIntensityAnalyzer()

In [54]:
text = 'My number one social media in the world'
score_dict = vds.polarity_scores(str(text))
print(score_dict)

{'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'compound': 0.0772}


# Import dataset

In [24]:
df_subjectivity_labelled = pd.read_csv('./df_english_subjectivity.csv', encoding='latin1') 
df_subjectivity_labelled.head(-1)

,Unnamed: 0,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,is_english,subjectivity,predicted subjectivity
0,1,3577f7a1-3394-4e77-813d-095a82cf8bcf,Kemar Richardson,https://play-lh.googleusercontent.com/a-/ACNPE...,Great,5,0,26.3.4,10/4/2022 20:32,NaN,NaN,True,OPINIONATED,OPINIONATED
1,2,7c8c56d9-d8ad-47d4-b24b-5289aa4529ff,Tracy Dunn,https://play-lh.googleusercontent.com/a/ALm5wu...,good,5,0,26.4.3,10/4/2022 20:31,NaN,NaN,True,OPINIONATED,OPINIONATED
2,3,80db804f-cccd-4b09-b690-abc12cbf0612,SG. Mugo. (Mugoz:),https://play-lh.googleusercontent.com/a-/ACNPE...,Good app,5,0,26.3.4,10/4/2022 20:30,NaN,NaN,True,OPINIONATED,OPINIONATED
3,4,4ed35e90-0f45-4865-81c4-b3a6f2ea49f7,Mwansa Judy,https://play-lh.googleusercontent.com/a-/ACNPE...,Most amazing app,5,0,26.3.4,10/4/2022 20:29,NaN,NaN,True,OPINIONATED,OPINIONATED
4,5,bd35bbe9-73c9-4e17-acaf-7aa1a71caed3,Muhammad Sajid,https://play-lh.googleusercontent.com/a-/ACNPE...,full Entertainment,4,0,25.9.4,10/4/2022 20:28,NaN,NaN,True,OPINIONATED,OPINIONATED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,1165,0799cf8d-a32e-4eea-9d1b-c93837532d0f,Steven Ishee,https://play-lh.googleusercontent.com/a-/ACNPE...,The button for linked Instagram accounts disap...,1,3,NaN,10/4/2022 8:05,NaN,NaN,True,OPINIONATED,OPINIONATED
995,1166,0421c7bf-cee8-4bd7-b692-aed487999c6c,ivan mendoza,https://play-lh.googleusercontent.com/a-/ACNPE...,Very good,4,0,26.4.3,10/4/2022 8:05,NaN,NaN,True,OPINIONATED,OPINIONATED
996,1167,346a2258-295c-464a-a9d8-a1709e5c1ae2,Jathol Gaming,https://play-lh.googleusercontent.com/a-/ACNPE...,hey TikTok why i get few views on my every vid...,1,0,26.4.3,10/4/2022 7:59,NaN,NaN,True,OPINIONATED,OPINIONATED
997,1168,20f1f6d2-eac9-42c1-bc8c-a205a0ea0484,Taneisha West,https://play-lh.googleusercontent.com/a/ALm5wu...,Everyone can see your Google account name and ...,5,0,19.5.42,10/4/2022 7:59,NaN,NaN,True,NEUTRAL,NEUTRAL


In [25]:
len(df_subjectivity_labelled['content'])

1000

# Vader subjectivity prediction
#### Data cleaning is NOT required to apply Vader. Punctuation and cases are important factors for Vader to detect text emotion

In [27]:
labels = []
for review in df_subjectivity_labelled['content']:
    score_dict = vds.polarity_scores(str(review))
    print(str(review))
    label_score = score_dict['compound']
    print(label_score)
    
    #decide label according to label_score:
    #positive: compound score >= 0.05
    #neutral: (compound score > -0.05) and (compound score < 0.05)
    #negative: compound score <= -0.05    
    if label_score >= 0.05:
        label = 'POSITIVE'
    elif (label_score > -0.05) and (label_score < 0.05):
        label = 'NEUTRAL'
    else:
        label = 'NEGATIVE'
    labels.append(label)

Great
0.6249
good
0.4404
Good app
0.4404
Most amazing app
0.624
full Entertainment
0.4215
Nice app
0.4215
Super and good app
0.7783
TikTok warning and video views problem in my id manimeraj9050 server Nepal tiktok down
-0.6249
Tiktok is a nice app
0.4215
Need my tiktok open note 
0.0
I enjoy it seeing some of videos and following friends
0.743
My favourite video
0.0
Wow
0.5859
I love tiktok its a great app
0.8519
I dont know whats going on anymore cause i cant download tiktok anymore i must download tiktok lite
0.0
Awesome app Ive ever had its the best to have
0.8519
I really like titok because its fun 
0.7264
When i make videos it keeps banneding my videosand i keep on starting making videos and account
0.0
TikTok Warning Problem in Bangladesh TikTok down
-0.6249
Amazing 
0.5859
Love it amazing
0.8402
Very good 
0.4927
Good
0.4404
Its an amazing app I havent seen any problem with it But for the last few days m facing an issue its not showing capcut edit effects
0.4641
My stress killer

In [29]:
df_subjectivity_labelled['label'] = labels
df_subjectivity_labelled.head()

,Unnamed: 0,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,is_english,subjectivity,predicted subjectivity,label
0,1,3577f7a1-3394-4e77-813d-095a82cf8bcf,Kemar Richardson,https://play-lh.googleusercontent.com/a-/ACNPE...,Great,5,0,26.3.4,10/4/2022 20:32,NaN,NaN,True,OPINIONATED,OPINIONATED,POSITIVE
1,2,7c8c56d9-d8ad-47d4-b24b-5289aa4529ff,Tracy Dunn,https://play-lh.googleusercontent.com/a/ALm5wu...,good,5,0,26.4.3,10/4/2022 20:31,NaN,NaN,True,OPINIONATED,OPINIONATED,POSITIVE
2,3,80db804f-cccd-4b09-b690-abc12cbf0612,SG. Mugo. (Mugoz:),https://play-lh.googleusercontent.com/a-/ACNPE...,Good app,5,0,26.3.4,10/4/2022 20:30,NaN,NaN,True,OPINIONATED,OPINIONATED,POSITIVE
3,4,4ed35e90-0f45-4865-81c4-b3a6f2ea49f7,Mwansa Judy,https://play-lh.googleusercontent.com/a-/ACNPE...,Most amazing app,5,0,26.3.4,10/4/2022 20:29,NaN,NaN,True,OPINIONATED,OPINIONATED,POSITIVE
4,5,bd35bbe9-73c9-4e17-acaf-7aa1a71caed3,Muhammad Sajid,https://play-lh.googleusercontent.com/a-/ACNPE...,full Entertainment,4,0,25.9.4,10/4/2022 20:28,NaN,NaN,True,OPINIONATED,OPINIONATED,POSITIVE


## Match labels to subjectivity

In [39]:
subjectivity = []

for label in df_subjectivity_labelled['label']: 
    if label == "POSITIVE" or label == "NEGATIVE":
        subjectivity.append("OPINIONATED")
    else:
        subjectivity.append("NEUTRAL")

In [40]:
df_subjectivity_labelled['subjectivity'] = subjectivity
df_subjectivity_labelled['subjectivity'].to_csv('./subjectivity.csv', sep='\t')

In [41]:
df_subjectivity = pd.read_csv('./df_english_subjectivity.csv', encoding='latin1') 
df_subjectivity[['content', 'subjectivity', 'predicted subjectivity']].head(20)

,content,subjectivity,predicted subjectivity
0,Great,OPINIONATED,OPINIONATED
1,good,OPINIONATED,OPINIONATED
2,Good app,OPINIONATED,OPINIONATED
3,Most amazing app,OPINIONATED,OPINIONATED
4,full Entertainment,OPINIONATED,OPINIONATED
5,Nice app,OPINIONATED,OPINIONATED
6,Super and good app,OPINIONATED,OPINIONATED
7,TikTok warning and video views problem in my i...,OPINIONATED,OPINIONATED
8,Tiktok is a nice app,OPINIONATED,OPINIONATED
9,Need my tiktok open note,NEUTRAL,NEUTRAL


## Model Evaluation

In [42]:
correct = 0

for i in range(1000):
    if df_subjectivity['subjectivity'][i] == df_subjectivity['predicted subjectivity'][i]:
        correct+=1
print(correct)  

862


In [43]:
print("prediction accurary: ", correct/1000)

prediction accurary:  0.862
